## CMPN403 Programming Assignment
Team:
- Yomna Osama Hussien 1190203
- Mohamed Hassan Mohamed 1190118

In [1]:
import re
import json
import graphviz
from typing import *
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### Utility functions

In [2]:
def verify_regex(regex: str) -> bool:
    try:
        re.compile(regex)
        return True
    except re.error:
        return False

def json_write(states: Dict[str, Dict], filename: str) -> None:
    if not filename.endswith('.json'):
        filename += '.json'
        
    with open(filename, 'w') as file:
        json.dump(states, file, indent=4)

def draw_nfa(json_file: str) -> graphviz.Digraph:
    with open(json_file, 'r') as file:
        nfa: Dict[str, Dict] = json.load(file)

    dot = graphviz.Digraph(comment='NFA')

    # add invisible starting state
    dot.node('startingStateH', 'startingStateH', style='invis')
    
    # add nodes
    for key in nfa.keys():
        if key == 'startingState':
            continue
        if nfa[key]['isTerminatingState']:
            dot.node(key, key, shape='doublecircle')
        else:
            dot.node(key, key)
    
    # add edges
    for key in nfa.keys():
        if key == 'startingState':
            continue
        for symbol in nfa[key].keys():
            if symbol == 'isTerminatingState':
                continue
            for next_state in nfa[key][symbol]:
                sy = symbol
                if symbol == 'epsilon':
                    sy = 'ε'
                dot.edge(key, next_state, label=sy)
    
    dot.edge('startingStateH', nfa['startingState'])

    return dot

def display_and_save_image(json_file: str, fileName: str) -> None:
    dot = draw_nfa(json_file)
    dot.format = 'png'
    dot.render(fileName)
    
    img = mpimg.imread(fileName+'.png')
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.figure(figsize=(15, 15))
    plt.show()

### Regex NFA
TODOs:
- Ranges
- ('*', '+', '?')


In [19]:
class NFA:
    def __init__(self) -> None:
        self.start_state = 0
        self.next_state = 0
        self.prev_state = 0
        self.prev_start = 0
        self.states = {"startingState": "S0", "S0": {"isTerminatingState": False}}

    def generate_NFA(self, regex: str, filename: str) -> None:
        if not verify_regex(regex):
            print("Invalid regex")
            return

        i = self.build_NFA(regex)
        
        if i == len(regex):
            self.states["S" + str(self.next_state)]["isTerminatingState"] = True

        json_write(self.states, filename)
    
    def build_NFA(self, regex: str) -> int:
        i = 0
        while i < len(regex):
            if regex[i] == '(':
                depth = 1
                subregex = regex[i+1:]
                for j in range(len(subregex)):
                    if subregex[j] == '(':
                        depth += 1
                    elif subregex[j] == ')':
                        depth -= 1
                    if depth == 0:
                        subregex = subregex[:j]
                        break
                sub_start_state = self.next_state + 1
                sub_next_state = self.next_state + 1
                sub_prev_state = self.next_state + 1
                sub_prev_start = self.next_state + 1
                
                self.states.update({"S" + str(sub_start_state + 1): {"isTerminatingState": False}})
                self.build_NFA(subregex)

                i += len(subregex) + 2
            elif regex[i] == '|':
                alt_start_state = self.next_state + 1
                alt_next_state = self.next_state + 1
                alt_prev_state = self.next_state + 1
                alt_prev_start = self.next_state + 1

                # Add ε-transitions from the current start state to the starting states of both branches
                self.states["S" + str(self.start_state)]["ε"].extend(["S" + str(alt_start_state), "S" + str(alt_start_state + 1)])

                # Build NFA for the first alternative
                self.start_state = alt_start_state
                self.next_state = alt_next_state
                self.prev_state = alt_prev_state
                self.prev_start = alt_prev_start
                self.build_NFA(regex[:i])

                # Build NFA for the second alternative
                self.start_state = alt_start_state + 1
                self.next_state = alt_next_state + 1
                self.prev_state = alt_prev_state
                self.prev_start = alt_prev_start
                self.build_NFA(regex[i+1:])

                # Create a new terminating state
                self.next_state += 1
                self.states["S" + str(self.next_state)] = {"isTerminatingState": False}

                # Connect terminating states of both branches to the new terminating state
                self.states["S" + str(alt_next_state)]["ε"].append("S" + str(self.next_state))
                self.states["S" + str(alt_next_state + 1)]["ε"].append("S" + str(self.next_state))

                # Update the state indices
                self.start_state = alt_start_state
                self.next_state = self.next_state + 1
                self.prev_state = alt_prev_state
                self.prev_start = alt_prev_start

                i += 1
            else:
                self.start_state, self.next_state, self.states = self.add_state(regex, self.start_state, self.next_state, self.states, i)
                i += 1
        return i
    
    def add_state(self, regex: str, start_state: int, next_state: int, states: Dict[str, Dict], i: int) -> Tuple[int, int, Dict[str, Dict]]:
        next_state += 1
        if regex[i] in states["S" + str(start_state)]:
            states["S" + str(start_state)][regex[i]].append("S" + str(next_state))
        else:
            states["S" + str(start_state)][regex[i]] = ["S" + str(next_state)]
        
        states["S" + str(next_state)] = {"isTerminatingState": False}
        start_state = next_state

        return start_state, next_state, states



#### Test literals

In [20]:
nfa = NFA()
nfa.generate_NFA("yomna", "literal.json")
display_and_save_image("literal.json", "literal")

KeyError: 'S1'

In [18]:
nfa = NFA()
nfa.generate_NFA("aa|b", "Alternate.json")
display_and_save_image("Alternate.json", "Alternate")

KeyError: 'ε'